<a href="https://colab.research.google.com/github/clam004/case/blob/main/zeroshot_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# load dependencies into our python environment
%%capture
! pip install transformers[sentencepiece]

If you get an error like:

`Transport endpoint is not connected`

Goto Runtime on menubar and click on restart runtime option

In [1]:
# Mount to my google drive which is where ive stored the data
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/case
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/case
ACES.ipynb   __pycache__		       utils.py
data	     sol.ipynb			       zeroshot_finetune.ipynb
modelstates  T0_3B_dialog_summarization.ipynb


In [4]:
#sys libs
import os
import sys
import random
import time
import re
import json

#string manupulation libs
import re
import string

#data manupulation libs
import numpy as np
import pandas as pd

#plotting tools
from matplotlib import pyplot as plt 

#torch libs
import torch
print('torch.__version__', torch.__version__)
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler

print('torch.cuda.device_count()', torch.cuda.device_count())
print('torch.cuda.empty_cache()', torch.cuda.empty_cache())

#huggingface transformers
import transformers
print('transformers.__version__', transformers.__version__)
from transformers import set_seed
from transformers import GPT2Tokenizer, GPT2LMHeadModel

#seeds
set_seed(42)
np.random.seed(0)
random.seed(0)
torch.manual_seed(0)

#jupyter stuff
%load_ext autoreload
%autoreload 2

torch.__version__ 1.11.0+cu113
torch.cuda.device_count() 0
torch.cuda.empty_cache() None
transformers.__version__ 4.20.1


In [12]:
proportion_to_valiation = 0.3

#df = pd.read_excel('Training_Data.xlsx')
df = pd.ExcelFile('Training_Data.xlsx')

print(df.sheet_names)  # see all sheet names

df = df.parse('Emergency')

df=df.dropna()

df['Label']= df['Label'].astype(str)

#df['Label'].str.strip()

print(df['Label'].value_counts())

records = df.to_records(index=False)
result = list(records)

neutral_count = 0
emerg_count = 0
for r in result:
  if r[1].strip() == 'emergency':
    emerg_count += 1
  else:
    neutral_count +=1
print("neutral_count, emerg_count", neutral_count, emerg_count)

print("number of total samples", len(result))

random.shuffle(result)

val_list = result[:int(len(result)*proportion_to_valiation)]
train_list = result[int(len(result)*proportion_to_valiation):]

print('len(val_list)', len(val_list))
print(val_list[:8])
print(train_list[:8])

['Emergency', 'Discrimination', 'Lewd', 'Medical_Questions', 'Dialog', 'Sheet1']
neutral      82
emergency    76
neutral      41
Name: Label, dtype: int64
neutral_count, emerg_count 123 76
number of total samples 199
len(val_list) 59
[('Fire alarm is broken it keeps blaring, my son called technician', 'neutral'), ('i dreamt i ran into my favorite singer', 'neutral'), ('My neighbor is driving me crazy', 'neutral '), ('i cant live anymore i want to kill myself or die in my sleep', 'emergency'), ('After hearing her story I felt sick to my stomach', 'neutral'), ('We need to break in our newly purchase car', 'neutral'), ('my son has been hitting me', 'emergency'), ('I am going to puke if I see another burger', 'neutral ')]
[('My son almost punched me', 'emergency'), ('Father is abusive when I was a child. I am better now', 'neutral'), ('I will die if someone tickled me that way ', 'neutral '), ('I chew slowly  now , sometimes food gets stuck in my throat', 'neutral'), ('I love watching the 

123+76=199

123/199 = .618

In [ ]:
pretrained_model = 'gpt2-medium'#'gpt2-large'#'gpt2-xl'#'gpt2'#

cache_dir = os.path.join(
    "./modelstates/hugface_models/",
    pretrained_model,
)

print("cache_dir=", cache_dir)

model_save_path = os.path.join(
    "./modelstates/finetuned_models",
    pretrained_model,
)

print("model_save_path=", model_save_path)

tokenizer = GPT2Tokenizer.from_pretrained(
    pretrained_model,
    cache_dir=cache_dir,
)

model = GPT2LMHeadModel.from_pretrained(
    pretrained_model,
    cache_dir=cache_dir,
)

cache_dir= ./modelstates/hugface_models/gpt2-large
model_save_path= ./modelstates/finetuned_models/gpt2-large


In [ ]:
def end_punctuation(utter):
    
    if len(utter) > 0:
      if utter[-1] not in ["?","!","."]:
          utter+="."
        
    return utter

def get_prompt(
    query_text,
    label_text='',
    prefix = '###',
    example_symbol = '[Text]:', 
    label_symbol = '[Label]:', 
    join_separator = '', 
):
    
    detect_prompt = []
    
    detect_prompt.append(prefix)

    query_text = end_punctuation(query_text.strip())
    detect_prompt.append(example_symbol+query_text)

    label_text = end_punctuation(label_text.strip())
    detect_prompt.append(label_symbol+label_text)
    
    return join_separator.join(detect_prompt)

print(get_prompt(val_list[0][0]))
print(get_prompt(val_list[0][0],val_list[0][1]))

###[Text]:i just got run over by a car.[Label]:
###[Text]:i just got run over by a car.[Label]:emergency.


In [ ]:
class EmotionDataset(Dataset):

    def __init__(self, tuple_list, tokenizer, chunk_size = 256):

        self.input_ids = []
        self.attn_masks = []
        self.input_token_lens = []

        def prompt2tensors(tuple_list, i):

            prompt = get_prompt(
                query_text = tuple_list[i][0],
                label_text = tuple_list[i][1],
            )

            encodings_dict = tokenizer(prompt, return_tensors="pt")

            input_ids = encodings_dict['input_ids']    
            attention_mask = encodings_dict['attention_mask']
            input_token_len = input_ids.shape[1]

            return input_ids, attention_mask, input_token_len

        print('len(tuple_list)', len(tuple_list))
        
        for i in range(len(tuple_list)):

            unfilled_chunk = True

            input_ids = torch.zeros(1, 0)
            attention_mask = torch.zeros(1, 0)
            input_token_len = 0
            j = i

            while unfilled_chunk:

              input_ids_, attention_mask_, input_token_len_ = \
                prompt2tensors(tuple_list, j)
              self.input_token_lens.append(input_token_len_)

              input_token_len += input_token_len_
              input_ids = torch.cat([input_ids, input_ids_], dim=1)
              attention_mask = torch.cat([attention_mask, attention_mask_], dim=1)

              if input_token_len > chunk_size:
                input_ids = input_ids[:,:chunk_size]
                attention_mask = attention_mask[:,:chunk_size]
                unfilled_chunk = False
              else:
                j = random.randint(0, len(tuple_list)-1)

            self.input_ids.append(input_ids.long())
            self.attn_masks.append(attention_mask.long())

        print('mean', np.mean(self.input_token_lens), 'min', min(self.input_token_lens), 'max', max(self.input_token_lens))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

traindata = EmotionDataset(train_list, tokenizer, chunk_size = 64)

tokenizer.batch_decode(traindata.__getitem__(0)[0])

len(tuple_list) 108
mean 17.37250554323725 min 12 max 24


["###[Text]:I am having a heart attack.[Label]:emergency.###[Text]:My phone screen has cracked on it.[Label]:neutral.###[Text]:She's been giving me such a headache.[Label]:neutral.###[Text]:respond faster buddy or youll hurt my feelings.[Label"]

In [ ]:
batch_size = 2

# Take training samples in random order. 
train_dataloader = DataLoader(
    traindata,  # The training samples.
    sampler = RandomSampler(traindata), # Select batches randomly. 
    batch_size = batch_size # Trains with this batch size.
)

for batch_idx, samples in enumerate(train_dataloader):

      break

samples[0].squeeze(1).shape

torch.Size([2, 64])

In [ ]:
class BaseAgent(torch.nn.Module):

    def __init__(self, model, tokenizer):
        
        super().__init__()
        
        self.model = model
        self.tokenizer = tokenizer
        
        self.optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=0.00001,
            betas=(0.9, 0.98),
            eps=1e-9,
        )
        
        self.num_gpus = torch.cuda.device_count()
        
        if self.num_gpus > 1:
            self.model.parallelize()
        elif self.num_gpus == 1:
            self.gpu0 = torch.device('cuda:0')
            #self.model = self.model.cuda()
            self.model = self.model.to(self.gpu0)
            '''you can do .to(cuda0) with tensors to'''
            
        print('self.model.device', self.model.device)
        
        self.num_params = \
          sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        print("num_params", self.num_params/1e6, "Million Parameters")
        
    def get_response(self, prompt, max_len = 16, no_repeat_ngram_size = None):
        
        prompt_dic = self.tokenizer(prompt,return_tensors="pt")
        prompt_ids = prompt_dic.input_ids
        prompt_mask = prompt_dic.attention_mask
        prompt_len = prompt_ids.shape[1]
        
        if self.num_gpus > 0:
            prompt_ids = prompt_ids.to(self.model.device)
            prompt_mask = prompt_mask.to(self.model.device)
        
        prompt_len = prompt_ids.shape[1]
        
        output_ids = self.model.generate(
            prompt_ids,
            attention_mask = prompt_mask,
            max_length = prompt_len+max_len,
            no_repeat_ngram_size = no_repeat_ngram_size,
            pad_token_id=tokenizer.eos_token_id,
        )

        generated_text = self.tokenizer.batch_decode(output_ids)[0]
        
        return generated_text
    
    def memorize(self, prompt, num_epochs = 3):

        print('start training loop')

        """ This is a rudimentary training loop
        that will train the agent to learn one
        sequence, the prompt. With enough epochs, this should
        result in memorizing the sequence, which is why this
        class method was names memorize. 
        There is nothing returned because the model attribute is modified inplace. 
        Args:
            prompt (string): the text to be learned
            num_epochs (int): the number of times we cycle though the training data, only 1 sample in this case
        """

        prompt_dic = self.tokenizer(prompt,return_tensors="pt")
        prompt_ids = prompt_dic.input_ids
        prompt_mask = prompt_dic.attention_mask
        prompt_len = prompt_ids.shape[1]

        if self.num_gpus > 0:
            prompt_ids = prompt_ids.to(self.model.device)
            prompt_mask = prompt_mask.to(self.model.device)
            
        source_ids = prompt_ids[:,:-1]
        target_ids = prompt_ids[:,1:]
        source_mask = prompt_mask[:,:-1]
        target_mask = prompt_mask[:,1:]

        # allow params to be updated
        self.model.train()

        for e in range(num_epochs):

            outputs = self.model(
                input_ids = prompt_ids,
                labels = prompt_ids,
                attention_mask = prompt_mask,
                token_type_ids=None,
            )

            # used logits and target tokens to calculate the loss
            loss = outputs.loss
            logits = outputs.logits

            # backward pass
            self.optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()

            print("epoch", e, "loss", loss.item())

        torch.cuda.empty_cache()

    
    def train(self, train_dataloader, num_epochs = 3):

      self.model.train()

      for epoch in range(num_epochs):

        for batch_idx, samples in enumerate(train_dataloader):

          samples = [s.to(self.model.device) for s in samples]

          outputs = self.model(
              input_ids = samples[0].squeeze(1),
              labels = samples[0].squeeze(1),
              attention_mask = samples[1].squeeze(1),
              token_type_ids=None,
          )

          # used logits and target tokens to calculate the loss
          loss = outputs.loss
          logits = outputs.logits

          # backward pass
          self.optimizer.zero_grad()
          loss.backward()
          torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
          self.optimizer.step()

        print("epoch", epoch, "loss", loss.item())

      torch.cuda.empty_cache()

    def eval(self, tuple_list, few_shot_list):

      f = ''

      for t in range(len(few_shot_list)):

        w = get_prompt(
            query_text = few_shot_list[t][0],
            label_text = few_shot_list[t][1],
        )

        f += w

      self.model.eval()

      accuracy_list = []

      for i in range(len(tuple_list)):

        x = get_prompt(
            query_text = tuple_list[i][0],
        )

        x = f+x

        y = tuple_list[i][1].strip()

        y_hat = self.get_response(x)

        y_hat = y_hat[len(x):].strip()

        if '.' in y_hat:
          y_hat = y_hat[:y_hat.index('.')]

        accuracy_list.append(y == y_hat)
        '''
        print(' ')
        print(x)
        print('_____________')
        print(y_hat)
        print('_____________')
        print(y)
        '''
        #break
      print(' ')
      print(x)
      print('_____________')
      print(y_hat)
      print('_____________')
      print(y)
      return np.mean(accuracy_list), accuracy_list

agent = BaseAgent(model, tokenizer)

self.model.device cuda:0
num_params 774.03008 Million Parameters


In [ ]:
acc, exam = agent.eval(val_list, train_list[:4])
acc

 
###[Text]:I am having a heart attack.[Label]:emergency.###[Text]:my nurse hasn't come yet and it's been an hour.[Label]:emergency.###[Text]:I fell asleep on the couch.[Label]:neutral.###[Text]:my blood sugar is very high today.[Label]:emergency.###[Text]:I cut myself while cooking and its bleeding.[Label]:
_____________
emergency
_____________
emergency


0.5434782608695652

In [ ]:
agent.train(train_dataloader, num_epochs = 3)

RuntimeError: ignored

In [ ]:
agent.eval(val_list)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_____________
____
_____________
neutral


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_____________
____
_____________
emergency


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_____________
____
_____________
emergency


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


_____________
vernacular
_____________
emergency
_____________
____
_____________
emergency


(0.0,
 [False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False])